In [ ]:
from datetime import datetime as dt, timedelta
from functools import reduce

import holoviews as hv
import geoviews as gv
import numpy as np
import panel as pn
from pyxlma import coords
from scipy.interpolate import griddata
import xarray as xr

from goes2go import GOES

from glmtools.io.lightning_ellipse import lightning_ellipse_rev

gv.extension('bokeh')

In [ ]:
date_i_want = dt(2022, 6, 2, 0, 0, 0)

In [ ]:
sbfobs = xr.open_dataset(f'/Volumes/LtgSSD/tobac_saves/tobac_Save_{date_i_want.strftime("%Y%m%d")}/seabreeze-obs.zarr', engine='zarr')

In [ ]:
has_lightning_mask = sbfobs.feature_flash_count.data > 0
has_zdr_mask = sbfobs.feature_zdrvol.data > 0
has_kdp_mask = sbfobs.feature_kdpvol.data > 0

nothing_mask = ~has_lightning_mask & ~has_zdr_mask & ~has_kdp_mask
zdr_only_mask = ~has_lightning_mask & has_zdr_mask & ~has_kdp_mask
kdp_only_mask = ~has_lightning_mask & ~has_zdr_mask & has_kdp_mask
zdr_kdp_mask = ~has_lightning_mask & has_zdr_mask & has_kdp_mask
zdr_lightning_mask = has_lightning_mask & has_zdr_mask & ~has_kdp_mask
kdp_lightning_mask = has_lightning_mask & ~has_zdr_mask & has_kdp_mask
everything_mask = has_lightning_mask & has_zdr_mask & has_kdp_mask
lightning_only_mask = has_lightning_mask & ~has_kdp_mask & ~has_zdr_mask

point_descriptor = {
    'Nothing': {'mask' : nothing_mask, 'color': 'blue', 'marker' : 'circle'},#'marker' : 'circle'},
    'ZDR' : {'mask' : zdr_only_mask, 'color' : 'orange', 'marker' : 'circle'},#'marker' : 'cross'},
    'KDP' : {'mask' : kdp_only_mask, 'color' : 'green', 'marker' : 'circle'},#'marker' : 'triangle'},
    'ZDR_KDP' : {'mask' : zdr_kdp_mask, 'color' : 'red', 'marker' : 'circle'},#'marker' : 'x'},
    'ZDR_Lightning' : {'mask' : zdr_lightning_mask, 'color' : 'purple', 'marker' : 'circle'},#'marker' : 'y'},
    'KDP_Lightning' : {'mask' : kdp_lightning_mask, 'color' : 'saddlebrown', 'marker' : 'circle'},#'marker' : 'hex'},
    'ZDR_KDP_Lightning' : {'mask' : everything_mask, 'color' : 'magenta', 'marker' : 'circle'},#'marker' : 'inverted_triangle'},
    'Lightning' : {'mask' : lightning_only_mask, 'color' : 'gray', 'marker' : 'circle'},#'marker' : 'square'},
}

In [ ]:
def plot_points(sbfobs, label, infodict, is_visible, filter_track_id=False, track_id=None):
    points_to_plot = sbfobs.isel(feature=infodict['mask'])
    if filter_track_id and track_id is not None:
        track_mask = (points_to_plot.feature_parent_track_id == track_id)
        points_to_plot = points_to_plot.isel(feature=track_mask)
    this_map = gv.Points((points_to_plot.feature_lon, points_to_plot.feature_lat, points_to_plot.feature, points_to_plot.feature_parent_track_id,
                          points_to_plot.feature_time.astype(str), points_to_plot.feature_kdpvol, points_to_plot.feature_zdrvol, points_to_plot.feature_flash_count,
                          points_to_plot.feature_echotop, (points_to_plot.feature_min_L2_MCMIPC-273.15)),
                         kdims=['Longitude', 'Latitude'],
                         vdims=['Feature ID', 'Track ID', 'Time', 'KDP Volume', 'ZDR Volume', 'Flash Count', 'Echo Top', 'CTT'],
                         label=label).opts(marker=infodict['marker'], color=infodict['color'], tools=['hover'], size=5, width=600, height=600, visible=is_visible)
    return this_map

In [ ]:
def plot_segmentation_scattter(sbfobs, feature_id, is_visible=False):
    if is_visible == False:
        return gv.Points(([])).opts(visible=False)
    else:
        this_feature = sbfobs.sel(feature=feature_id)
        this_feature_time_idx = this_feature.feature_time_index.data.item()
        this_seg_mask = this_feature.isel(time=this_feature_time_idx).segmentation_mask.transpose('x', 'y')
        lats_of_feature = this_feature.lat.transpose('y', 'x').data.flatten()[this_seg_mask.data.flatten() == feature_id]
        lons_of_feature = this_feature.lon.transpose('y', 'x').data.flatten()[this_seg_mask.data.flatten() == feature_id]
        return gv.Points((lons_of_feature, lats_of_feature)).opts(marker='circle', color='black', size=5, width=600, height=600, visible=True)

In [ ]:
def plot_conceptual(sbfobs, track_id, is_visible=False):
    feature_mask = sbfobs.feature_parent_track_id == track_id
    this_track = sbfobs.sel(track=track_id).isel(feature=feature_mask)
    if is_visible and this_track.feature_time.data.shape[0] > 0:
        first_time = this_track.feature_time.data.min()
        last_time = this_track.feature_time.data.max()
        kdp_col_plot = hv.Curve((this_track.time, this_track.track_kdpvol), kdims=['Time'], vdims=['Volume'], label='Track KDP').opts(color='red', width=600, height=600, xlim=(first_time, last_time))
        kdp_col_scatter = hv.Scatter((this_track.feature_time, this_track.feature_kdpvol, this_track.feature), kdims=['Time'], vdims=['Volume', 'Feature ID'], label='Feature KDP').opts(color='red', width=600, height=600, tools=['hover'])
        zdr_col_plot = hv.Curve((this_track.time, this_track.track_zdrvol), kdims=['Time'], vdims=['Volume'], label='Track ZDR').opts(color='orange', width=600, height=600, xlim=(first_time, last_time))
        zdr_col_scatter = hv.Scatter((this_track.feature_time, this_track.feature_zdrvol, this_track.feature), kdims=['Time'], vdims=['Volume', 'Feature ID'], label='Feature ZDR').opts(color='orange', width=600, height=600, tools=['hover'])
        lightning_plot = hv.Curve((this_track.time, this_track.track_flash_count), kdims=['Time'], vdims=['Volume'], label='Track Lightning').opts(color='yellow', width=600, height=600)
        lightning_scatter = hv.Scatter((this_track.feature_time, this_track.feature_flash_count, this_track.feature), kdims=['Time'], vdims=['Flash Count', 'Feature ID'], label='Feature Lightning').opts(color='yellow', width=600, height=600, tools=['hover'])
        return kdp_col_plot * kdp_col_scatter * zdr_col_plot * zdr_col_scatter * lightning_plot * lightning_scatter
    else:
        return hv.Scatter((sbfobs.feature_time.data, sbfobs.feature.data)).opts(visible=False, width=600, height=600, tools=['hover']) * hv.Scatter((sbfobs.feature_time.data, sbfobs.feature.data)).opts(visible=False, width=600, height=600, tools=['hover'])

In [ ]:
goes_ctt = GOES(satellite=16, product='ABI-L2-MCMIPC')
goes_time_range_start = sbfobs.time.data.astype('datetime64[s]').astype(dt)[0]
goes_time_range_end = sbfobs.time.data.astype('datetime64[s]').astype(dt)[-1]
goes_ctt = GOES(satellite=16, product='ABI-L2-MCMIPC')
print('Start download')
download_results = goes_ctt.timerange(goes_time_range_start-timedelta(minutes=15), goes_time_range_end+timedelta(minutes=15), max_cpus=4)
print('End download')

download_results['valid'] = download_results[['start', 'end']].mean(axis=1)
valid_times = download_results['valid'].values.astype('datetime64[s]')

In [ ]:
def plot_satellite(sbfobs, feat_id, download_results, is_visible=True):
    if is_visible:
        print('plotting!')
        this_feature = sbfobs.sel(feature=feat_id)
        print('selected')
        this_feature_time_idx = this_feature.feature_time_index.data.item()
        print(f'found index: {this_feature_time_idx}')
        sat_to_plot = download_results[download_results['valid'].astype('datetime64[s]') == sbfobs.closest_satellite_time.data[this_feature_time_idx]]['file'].values[0]
        print('identified file')
        
        # Find the index boundaries of the feature
        x_indices_valid, y_indices_valid = np.asarray(sbfobs.isel(time=this_feature_time_idx).segmentation_mask == feat_id).nonzero()
        first_x_idx = np.min(x_indices_valid)
        first_y_idx = np.min(y_indices_valid)
        last_x_idx = np.max(x_indices_valid)
        last_y_idx = np.max(y_indices_valid)

        # Trim the grid to a rectangle surrounding the feature
        grid_x2d, grid_y2d = np.meshgrid(sbfobs.x, sbfobs.y)
        index_pad = 10
        first_x_idx = np.max([first_x_idx-index_pad, 0])
        first_y_idx = np.max([first_y_idx-index_pad, 0])
        last_x_idx = np.min([last_x_idx+index_pad, sbfobs.x.shape[0]-1])
        last_y_idx = np.min([last_y_idx+index_pad, sbfobs.y.shape[0]-1])
        this_feature_x2d = grid_x2d[first_x_idx:last_x_idx+1, first_y_idx:last_y_idx+1]
        this_feature_y2d = grid_y2d[first_x_idx:last_x_idx+1, first_y_idx:last_y_idx+1]
        this_feature_lon2d = sbfobs.lon.data[first_x_idx:last_x_idx+1, first_y_idx:last_y_idx+1]
        this_feature_lat2d = sbfobs.lat.data[first_x_idx:last_x_idx+1, first_y_idx:last_y_idx+1]
        this_goes_x = sbfobs.g16_scan_x.data[first_x_idx:last_x_idx+1, first_y_idx:last_y_idx+1]
        this_goes_y = sbfobs.g16_scan_y.data[first_x_idx:last_x_idx+1, first_y_idx:last_y_idx+1]
        this_feature_g16_xmax = np.max(this_goes_x)
        this_feature_g16_ymax = np.max(this_goes_y)
        this_feature_g16_xmin = np.min(this_goes_x)
        this_feature_g16_ymin = np.min(this_goes_y)
        feature_padding = .0015 # A little bit of padding to be safe
        satellite_data = xr.open_dataset('../'+sat_to_plot).sel(y=slice(this_feature_g16_ymax+feature_padding, this_feature_g16_ymin-feature_padding),
                                    x=slice(this_feature_g16_xmin-feature_padding, this_feature_g16_xmax+feature_padding))
        print('loaded data')
        sat_y2d, sat_x2d = np.meshgrid(satellite_data.y.data, satellite_data.x.data)
        sat_z2d = np.zeros_like(sat_x2d)
        ltg_ell = lightning_ellipse_rev[1]
        feat_echotop = 0
        satsys = coords.GeostationaryFixedGridSystem(subsat_lon=-75.,
                                                        sweep_axis='x', ellipse=(ltg_ell[0] - 14e3 + feat_echotop, ltg_ell[1] - 6e3 + feat_echotop))
        satellite_ECEF = satsys.toECEF(sat_x2d.flatten(), sat_y2d.flatten(), sat_z2d.flatten())
        tpcs = coords.TangentPlaneCartesianSystem(ctrLon=sbfobs.center_lon, ctrLat=sbfobs.center_lat, ctrAlt=0)
        sat_tpcs = tpcs.fromECEF(*satellite_ECEF)
        sat_tpcs_X = sat_tpcs[0].reshape(sat_x2d.shape)
        sat_tpcs_Y = sat_tpcs[1].reshape(sat_y2d.shape)
        print('transformed data')
        interp_vals = griddata(
            np.array([sat_tpcs_X.flatten(), sat_tpcs_Y.flatten()]).T,
            satellite_data.CMI_C13.data.T.flatten(),
            np.array([this_feature_x2d.flatten(), this_feature_y2d.flatten()]).T,
            method='linear'
        )
        print(this_feature_lon2d.shape)
        print(this_feature_lat2d.shape)
        print(interp_vals.shape)
        interp_vals = interp_vals.reshape(this_feature_lon2d.shape)
        print(interp_vals.shape)
        return gv.QuadMesh((this_feature_lon2d, this_feature_lat2d, interp_vals), kdims=['Longitude', 'Latitude'], vdims=['Brightness Temperature']).opts(cmap='viridis', colorbar=True, tools=['hover'], alpha=0.5)
    else:
        return gv.QuadMesh((sbfobs.lon, sbfobs.lat, sbfobs.segmentation_mask.isel(time=0))).opts(visible=False, colorbar=True, tools=['hover'], alpha=0.5)

In [ ]:
track_toggle = pn.widgets.Checkbox(name='Filter by Track ID', value=False)
track_select = pn.widgets.IntInput(name='Track ID', start=sbfobs.track.data.astype(int).min(), end=sbfobs.track.data.astype(int).max(), value=0, step=1)
show_nothing = pn.widgets.Checkbox(name='Show Nothing', value=True)
show_zdr = pn.widgets.Checkbox(name='Show ZDR', value=True)
show_kdp = pn.widgets.Checkbox(name='Show KDP', value=True)
show_zdr_kdp = pn.widgets.Checkbox(name='Show ZDR_KDP', value=True)
show_zdr_lightning = pn.widgets.Checkbox(name='Show ZDR_Lightning', value=True)
show_kdp_lightning = pn.widgets.Checkbox(name='Show KDP_Lightning', value=True)
show_zdr_kdp_lightning = pn.widgets.Checkbox(name='Show ZDR_KDP_Lightning', value=True)
show_lightning = pn.widgets.Checkbox(name='Show Lightning', value=True)
seg_toggle = pn.widgets.Checkbox(name='Show Segmentation', value=False)
seg_selector = pn.widgets.IntInput(name='Segmentation ID', start=1, end=sbfobs.feature.data.max(), value=0, step=1)
sat_toggle = pn.widgets.Checkbox(name='Show Satellite', value=False)

In [ ]:
nothing_map = gv.DynamicMap(pn.bind(plot_points, sbfobs, 'Nothing', point_descriptor['Nothing'], show_nothing, filter_track_id=track_toggle, track_id=track_select))
zdr_map = gv.DynamicMap(pn.bind(plot_points, sbfobs, 'ZDR', point_descriptor['ZDR'], show_zdr, filter_track_id=track_toggle, track_id=track_select))
kdp_map = gv.DynamicMap(pn.bind(plot_points, sbfobs, 'KDP', point_descriptor['KDP'], show_kdp, filter_track_id=track_toggle, track_id=track_select))
zdr_kdp_map = gv.DynamicMap(pn.bind(plot_points, sbfobs, 'ZDR_KDP', point_descriptor['ZDR_KDP'], show_zdr_kdp, filter_track_id=track_toggle, track_id=track_select))
zdr_lightning_map = gv.DynamicMap(pn.bind(plot_points, sbfobs, 'ZDR_Lightning', point_descriptor['ZDR_Lightning'], show_zdr_lightning, filter_track_id=track_toggle, track_id=track_select))
kdp_lightning_map = gv.DynamicMap(pn.bind(plot_points, sbfobs, 'KDP_Lightning', point_descriptor['KDP_Lightning'], show_kdp_lightning, filter_track_id=track_toggle, track_id=track_select))
zdr_kdp_lightning_map = gv.DynamicMap(pn.bind(plot_points, sbfobs, 'ZDR_KDP_Lightning', point_descriptor['ZDR_KDP_Lightning'], show_zdr_kdp_lightning, filter_track_id=track_toggle, track_id=track_select))
lightning_map = gv.DynamicMap(pn.bind(plot_points, sbfobs, 'Lightning', point_descriptor['Lightning'], show_lightning, filter_track_id=track_toggle, track_id=track_select))

seg_map = gv.DynamicMap(pn.bind(plot_segmentation_scattter, sbfobs, seg_selector, seg_toggle))

satellite_map = gv.DynamicMap(pn.bind(plot_satellite, sbfobs, seg_selector, download_results, sat_toggle))

map_plot = nothing_map * zdr_map * kdp_map * zdr_kdp_map * zdr_lightning_map * kdp_lightning_map * zdr_kdp_lightning_map * lightning_map * seg_map * satellite_map * gv.tile_sources.OSM

concept_plot = hv.DynamicMap(pn.bind(plot_conceptual, sbfobs, track_select, track_toggle))

In [ ]:
track_flash_count = sbfobs.track_flash_count.sum(dim='time')
track_zdrvol = sbfobs.track_zdrvol.sum(dim='time')
track_kdpvol = sbfobs.track_kdpvol.sum(dim='time')

track_ccn = sbfobs['track_ccn_profile_0.6'].isel(vertical_levels=0).mean(dim='time')
track_ecape = sbfobs.track_mlecape.mean(dim='time')
track_18et = sbfobs.track_echo_top.max(dim='time')
track_ctt = sbfobs.track_min_L2_MCMIPC.min(dim='time') - 273.15

In [ ]:
track_lightning_mask = track_flash_count > 0
track_zdr_mask = track_zdrvol > 0
track_kdp_mask = track_kdpvol > 0

track_nothing_mask = ~track_lightning_mask & ~track_zdr_mask & ~track_kdp_mask
track_zdr_only_mask = ~track_lightning_mask & track_zdr_mask & ~track_kdp_mask
track_kdp_only_mask = ~track_lightning_mask & ~track_zdr_mask & track_kdp_mask
track_zdr_kdp_mask = ~track_lightning_mask & track_zdr_mask & track_kdp_mask
track_zdr_lightning_mask = track_lightning_mask & track_zdr_mask & ~track_kdp_mask
track_kdp_lightning_mask = track_lightning_mask & ~track_zdr_mask & track_kdp_mask
track_everything_mask = track_lightning_mask & track_zdr_mask & track_kdp_mask
track_lightning_only_mask = track_lightning_mask & ~track_kdp_mask & ~track_zdr_mask

In [ ]:
track_point_descriptor = {
    'Nothing': {'mask' : track_nothing_mask, 'color': 'blue', 'marker' : 'circle'},#'marker' : 'circle'},
    'ZDR' : {'mask' : track_zdr_only_mask, 'color' : 'orange', 'marker' : 'circle'},#'marker' : 'cross'},
    'KDP' : {'mask' : track_kdp_only_mask, 'color' : 'green', 'marker' : 'circle'},#'marker' : 'triangle'},
    'ZDR_KDP' : {'mask' : track_zdr_kdp_mask, 'color' : 'red', 'marker' : 'circle'},#'marker' : 'x'},
    'ZDR_Lightning' : {'mask' : track_zdr_lightning_mask, 'color' : 'purple', 'marker' : 'circle'},#'marker' : 'y'},
    'KDP_Lightning' : {'mask' : track_kdp_lightning_mask, 'color' : 'saddlebrown', 'marker' : 'circle'},#'marker' : 'hex'},
    'ZDR_KDP_Lightning' : {'mask' : track_everything_mask, 'color' : 'magenta', 'marker' : 'circle'},#'marker' : 'inverted_triangle'},
    'Lightning' : {'mask' : track_lightning_only_mask, 'color' : 'gray', 'marker' : 'circle'},#'marker' : 'square'},
}

In [ ]:
ccn_plots = []
ecape_plots = []
et_plots = []
ctt_plots = []

for label, infodict in track_point_descriptor.items():
    this_ccn = track_ccn.isel(track=infodict['mask'])
    this_ecape = track_ecape.isel(track=infodict['mask'])
    this_et = track_18et.isel(track=infodict['mask'])
    this_ctt = track_ctt.isel(track=infodict['mask'])
    this_flash_count = track_flash_count.isel(track=infodict['mask'])

    this_ccn_plot = hv.Scatter((this_ccn, this_flash_count, this_ccn.track), kdims=['CCN'], vdims=['Flash Count', 'Track ID'], label=label).opts(color=infodict['color'], width=600, height=600, tools=['hover'], show_legend=False)
    this_ecape_plot = hv.Scatter((this_ctt, this_flash_count, this_ecape.track), kdims=['ECAPE'], vdims=['Flash Count', 'Track ID'], label=label).opts(color=infodict['color'], width=600, height=600, tools=['hover'], show_legend=False)
    this_et_plot = hv.Scatter((this_et, this_flash_count, this_et.track), kdims=['Echo Top'], vdims=['Flash Count', 'Track ID'], label=label).opts(color=infodict['color'], width=600, height=600, tools=['hover'], show_legend=False)
    this_ctt_plot = hv.Scatter((this_ctt, this_flash_count, this_ctt.track), kdims=['CTT'], vdims=['Flash Count', 'Track ID'], label=label).opts(color=infodict['color'], width=600, height=600, tools=['hover'], show_legend=False)

    ccn_plots.append(this_ccn_plot)
    ecape_plots.append(this_ecape_plot)
    et_plots.append(this_et_plot)
    ctt_plots.append(this_ctt_plot)

In [ ]:
layout = (reduce(lambda x, y: x * y, ccn_plots) + reduce(lambda x, y: x * y, ecape_plots) + reduce(lambda x, y: x * y, et_plots) + reduce(lambda x, y: x * y, ctt_plots))

In [ ]:
dash = pn.Column(pn.Row(
    pn.Column(map_plot),
    pn.Column(track_toggle, track_select, show_nothing, show_zdr, show_kdp, show_zdr_kdp, show_zdr_lightning, show_kdp_lightning, show_zdr_kdp_lightning, show_lightning, seg_toggle, seg_selector, sat_toggle),
    concept_plot
), layout)

In [ ]:
pn.serve(dash)

In [ ]:
sbfobs.feature.data[np.isnan(sbfobs.feature_min_L2_MCMIPC.data)]

In [ ]:
sbfobs.feature_area.data == 0

In [ ]:
np.sum(sbfobs.segmentation_mask.isel(time=0).data == 7)